# 그래프 임베딩 유사도 기반 추천 리스트를 DB에 저장

In [33]:
from graph.utils import *
import pymysql
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import h5py
import pickle
from annoy import AnnoyIndex

In [ ]:
# DB 접속
client = MongoClient("###")
db = client['final_project']
similarity = db['similarity']

In [35]:
# 데이터 확보
result = list(db.repository.find())

In [38]:
# graph 데이터를 담은 graph.txt파일 생성
with open('graph/graph.txt', 'w') as f:
    for ru in tqdm(result):
        try:
            for r in ru['star_user_list']:
                line = str(r) + '\t' + 'star' + '\t' + str(ru['rid']) + '\n'
                f.write(line)
        except:
            pass
    f.close

100%|██████████| 2698/2698 [00:01<00:00, 2264.71it/s]


In [39]:
ENTITY_MIN_COUNT = 1

In [40]:
# 모델 학습 시작
%time !torchbiggraph_import_from_tsv --lhs-col=0 --rel-col=1 --rhs-col=2 --entity-min-count={ENTITY_MIN_COUNT} graph/graph_config.py graph/graph.txt

[2022-05-30 21:30:14.920885] Looking up relation types in the edge files...
[2022-05-30 21:30:16.017044] - Found 1 relation types
[2022-05-30 21:30:16.017091] - Removing the ones with fewer than 1 occurrences...
[2022-05-30 21:30:16.017151] - Left with 1 relation types
[2022-05-30 21:30:16.017164] - Shuffling them...
[2022-05-30 21:30:16.017198] Searching for the entities in the edge files...
[2022-05-30 21:30:17.747215] Entity type all:
[2022-05-30 21:30:17.747263] - Found 338884 entities
[2022-05-30 21:30:17.747278] - Removing the ones with fewer than 1 occurrences...
[2022-05-30 21:30:17.876718] - Left with 338884 entities
[2022-05-30 21:30:17.876778] - Shuffling them...
[2022-05-30 21:30:18.235145] Preparing counts and dictionaries for entities and relation types:
[2022-05-30 21:30:18.235381] - Writing count of entity type all and partition 0
[2022-05-30 21:30:18.286763] - Writing count of entity type all and partition 1
[2022-05-30 21:30:18.339943] - Writing count of entity type a

In [41]:
for TEMPLATE in ['checkpoint_version.txt', 'embeddings_all_*.h5', 'model.v*.h5']:
    !rm -f graph/data/graph_edges/{TEMPLATE}

In [42]:
%time !torchbiggraph_train graph_config.py -p edge_paths=graph/data/graph_edges/graph-train_partitioned

2022-05-30 21:30:43,207   [Trainer-0] Loading entity counts...
2022-05-30 21:30:43,208   [Trainer-0] Creating workers...
2022-05-30 21:30:43,288   [Trainer-0] Initializing global model...
2022-05-30 21:30:43,291   [Trainer-0] Starting epoch 1 / 10, edge path 1 / 1, edge chunk 1 / 1
2022-05-30 21:30:43,292   [Trainer-0] Edge path: ./data/mysql-2016-01-08/graph_edges/graph-train_partitioned
2022-05-30 21:30:43,292   [Trainer-0] still in queue: 15
2022-05-30 21:30:43,293   [Trainer-0] Swapping partitioned embeddings None ( 3 , 3 )
2022-05-30 21:30:43,293   [Trainer-0] Loading partitioned embeddings from checkpoint
2022-05-30 21:30:45,676   [Trainer-0] ( 3 , 3 ): Training stats: loss:  10.0111 , reg:  0 , violators_lhs:  70.2874 , violators_rhs:  72.7735 , count:  65849
2022-05-30 21:30:45,676   [Trainer-0] ( 3 , 3 ): bucket 1 / 16 : Trained 65849 edges in 2.33 s ( 0.028 M/sec ); Eval 2*0 edges in 0.00 s ( 0 M/sec ); io: 0.05 s for 35,468,776 bytes ( 681.66 MB/sec )
2022-05-30 21:30:45,676

In [43]:
path = f'graph/data/graph_edges'
files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.find("embeddings_all") != -1])
files = files[0:2] + files[8:] + files[2:8]
files

['./data/mysql-2016-01-08//graph_edges/embeddings_all_0.v10.h5',
 './data/mysql-2016-01-08//graph_edges/embeddings_all_1.v10.h5',
 './data/mysql-2016-01-08//graph_edges/embeddings_all_2.v10.h5',
 './data/mysql-2016-01-08//graph_edges/embeddings_all_3.v10.h5']

In [44]:
embeddings=[]
for file in tn(files):
    with h5py.File(file, "r") as hf:
        embedding = hf["embeddings"][...]
    embeddings.append(embedding)
embeddings = np.concatenate(embeddings, axis=0)
embeddings.shape

/tmp/ipykernel_26095/2645363881.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tn(files):


  0%|          | 0/4 [00:00<?, ?it/s]

(338884, 100)

In [45]:
!torchbiggraph_export_to_tsv graph/graph_config.py --entities-output {DATA_FOLDER}/graph_edges/embeddings.tsv --relation-types-output {DATA_FOLDER}/graph_edges/relation_types_parameters.tsv

Loading relation types and entities...
Initializing model...
Loading model check point...
Writing entity embeddings...
Reading embeddings for entity type all partition 0 from checkpoint...
Writing embeddings for entity type all partition 0 to output file...
- Processed 5000/84721 entities so far...
- Processed 10000/84721 entities so far...
- Processed 15000/84721 entities so far...
- Processed 20000/84721 entities so far...
- Processed 25000/84721 entities so far...
- Processed 30000/84721 entities so far...
- Processed 35000/84721 entities so far...
- Processed 40000/84721 entities so far...
- Processed 45000/84721 entities so far...
- Processed 50000/84721 entities so far...
- Processed 55000/84721 entities so far...
- Processed 60000/84721 entities so far...
- Processed 65000/84721 entities so far...
- Processed 70000/84721 entities so far...
- Processed 75000/84721 entities so far...
- Processed 80000/84721 entities so far...
- Processed all 84721 entities
Reading embeddings for e

In [46]:
# 임베딩 파일 불러오기
embeddings = pd.read_csv('graph/data/graph_edges/embeddings.tsv', sep='\t',header=None,index_col=0)

In [47]:
# 그래프 데이터 불러오기
entities = pd.read_csv('graph/graph.txt', sep='\t',header=None)

In [48]:
# 유저 목록과 repo 목록
users = entities[0].unique()
repos = entities[2].unique()

In [49]:
# repo 임베딩만 추출
repo_embs = embeddings[np.isin(embeddings.index, repos)==True]

In [51]:
repos_ann = AnnoyIndex(100, 'angular')
repos_mapping = {}
for i, repo_id in enumerate(repo_embs.index):
    repos_ann.add_item(i, np.array(repo_embs.loc[repo_id]))
    repos_mapping[repo_id] = i

repos_ann.build(1000)
repos_ann.save('graph/repos.annoy')
pd.to_pickle(repos_mapping, 'graph/repos_mapping.pkl')

In [52]:
repos_mapping = pd.read_pickle('graph/repos_mapping.pkl')
repos_mapping_rev = {v: k for k, v in repos_mapping.items()}
repos_ann = AnnoyIndex(100, 'angular')
repos_ann.load('graph/repos.annoy')

True

In [53]:
# repo별로 유사한 repo 10개씩 
graph_sim = {}
for repo_id in tqdm(repo_embs.index):
    graph_sim[repo_id] = [repos_mapping_rev[i] for i in repos_ann.get_nns_by_item(repos_mapping[repo_id],11)[1:]]

100%|██████████| 2688/2688 [00:03<00:00, 687.63it/s]


In [ ]:
# DB에 저장
sim_list = []
for rid in tqdm(graph_sim.keys()):
    temp_dic = {}
    temp_dic['rid'] = rid
    temp_dic['graph_sim'] = graph_sim[rid]
    sim_list.append(temp_dic)

similarity.insert_many(sim_list)